In [1]:
import sys
sys.path.append("/home/ece/utkarsh/NLTM/")

In [2]:
import torch
from interlingua_model.model.modelling_mbart_modified_pos_dp import MBartForConditionalGeneration
from interlingua_model.data_handling.datacollater_pos_dp import DataCollatorForSeq2Seq

/home/ece/anaconda3/envs/ni-ut/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import wandb
wandb.init(project="NLTM_POS", entity="divyanshuusingh", mode = "disabled")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [4]:
torch.cuda.is_available()

True

In [5]:
from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          #DataCollatorForSeq2Seq,
                          Seq2SeqTrainingArguments,
                          Seq2SeqTrainer)


In [6]:
model_name = "ai4bharat/IndicBART"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, use_fast=False, keep_accents=True)
#

In [7]:
model = MBartForConditionalGeneration.from_pretrained(pretrained_model_name_or_path = "/home/ece/utkarsh/old_source_interlingua-model/")

Some weights of MBartForConditionalGeneration were not initialized from the model checkpoint at /home/ece/utkarsh/old_source_interlingua-model/ and are newly initialized: ['model.encoder.embed_positions_pos.weight', 'model.encoder.embed_positions_deprel.weight', 'model.encoder.embed_positions_pp.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
#model.__dict__

# seeing class MBartEncoder(MBartPreTrainedModel) 

In [9]:
model # yeah

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(64014, 1024, padding_idx=0)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(64014, 1024, padding_idx=0)
      (embed_positions_pos): Embedding(64014, 1024, padding_idx=0)
      (embed_positions_deprel): Embedding(64014, 1024, padding_idx=0)
      (embed_positions_pp): Embedding(64014, 1024, padding_idx=0)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0): MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_f

In [10]:
#model._modules['model']._modules['encoder']

In [11]:
model._modules['model']._modules['encoder'].embed_positions

MBartLearnedPositionalEmbedding(1026, 1024)

In [12]:
""" 
lets write mbartposembedding---

Q1. what input is model taking and then giving to function 

"""

' \nlets write mbartposembedding---\n\nQ1. what input is model taking and then giving to function \n\n'

# import dataset cdac

In [13]:
from datasets import load_from_disk

In [14]:
tok_data = load_from_disk('/home/ece/utkarsh/syn_data')

In [15]:
tok_data

DatasetDict({
    train: Dataset({
        features: ['Hindi', 'Sanskrit', 'input_ids', 'token_type_ids', 'attention_mask', 'labels', 'pos', 'deprel', 'pp'],
        num_rows: 11565
    })
    test: Dataset({
        features: ['Hindi', 'Sanskrit', 'input_ids', 'token_type_ids', 'attention_mask', 'labels', 'pos', 'deprel', 'pp'],
        num_rows: 1286
    })
})

# training

In [16]:
from interlingua_model.trainer.trainer_seq2seq_pos_dp import Seq2SeqTrainer
from interlingua_model.trainer import trainer_pos_dp
from interlingua_model.data_handling.datacollater_pos_dp import DataCollatorForSeq2Seq

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)  

In [18]:
from transformers import (AutoTokenizer,
                          AutoModelForSeq2SeqLM,
                          #DataCollatorForSeq2Seq,
                          Seq2SeqTrainingArguments
                          )
import numpy as np

In [19]:
batch_size = 16

In [20]:
args = Seq2SeqTrainingArguments(
            'translation_pos_dp',
            evaluation_strategy='epoch',
            learning_rate=0.001,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            weight_decay=0.0001,
            save_total_limit=2,
            num_train_epochs=20,
            #label_names = ["pos"],
            predict_with_generate=True,
            report_to="wandb")

In [21]:
args.__dict__

{'output_dir': 'translation_pos_dp',
 'overwrite_output_dir': False,
 'do_train': False,
 'do_eval': True,
 'do_predict': False,
 'evaluation_strategy': <IntervalStrategy.EPOCH: 'epoch'>,
 'prediction_loss_only': False,
 'per_device_train_batch_size': 16,
 'per_device_eval_batch_size': 16,
 'per_gpu_train_batch_size': None,
 'per_gpu_eval_batch_size': None,
 'gradient_accumulation_steps': 1,
 'eval_accumulation_steps': None,
 'eval_delay': 0,
 'learning_rate': 0.001,
 'weight_decay': 0.0001,
 'adam_beta1': 0.9,
 'adam_beta2': 0.999,
 'adam_epsilon': 1e-08,
 'max_grad_norm': 1.0,
 'num_train_epochs': 20,
 'max_steps': -1,
 'lr_scheduler_type': <SchedulerType.LINEAR: 'linear'>,
 'warmup_ratio': 0.0,
 'warmup_steps': 0,
 'log_level': -1,
 'log_level_replica': -1,
 'log_on_each_node': True,
 'logging_dir': 'translation_pos_dp/runs/Sep26_11-58-24_ece-System',
 'logging_strategy': <IntervalStrategy.STEPS: 'steps'>,
 'logging_first_step': False,
 'logging_steps': 500,
 'logging_nan_inf_filter

In [22]:
tokenized_datasets = tok_data

In [23]:
from datasets import load_metric

In [24]:
metric = load_metric('sacrebleu')

In [25]:
def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [[label.strip()] for label in labels]

        return preds, labels

def compute_metrics(eval_preds):
        #print("compute_met called")
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {'bleu': result['score']}

        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        result['gen_len'] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}
        #print("compute-met-end")
        return result


In [26]:
trainer = Seq2SeqTrainer(
              model,
              args,
              train_dataset=tokenized_datasets['train'],
              eval_dataset=tokenized_datasets['test'],
              data_collator=data_collator,
              tokenizer=tokenizer,
              compute_metrics=compute_metrics)  



In [27]:
trainer.train()

/home/ece/anaconda3/envs/ni-ut/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/home/ece/anaconda3/envs/ni-ut/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,2.319595,9.358500,19.759700


## evaluate after training

In [ ]:
max_target_length = 128


In [ ]:
trainer.evaluate()

predict_dataset = tokenized_datasets['test']
predict_results = trainer.predict(predict_dataset,
                                    metric_key_prefix='predict',
                                    max_length=max_target_length)

predictions = tokenizer.batch_decode(predict_results.predictions,
                                        skip_special_tokens=True,
                                        clean_up_tokenization_spaces=True)

predictions = [pred.strip() for pred in predictions]

# input_sentences = tokenizer.batch_decode(tokenized_datasets['test']['input_ids'], skip_special_tokens=True)
# output_sentences = tokenizer.batch_decode(tokenized_datasets['test']['labels'], skip_special_tokens=True)


In [ ]:
predictions

In [ ]:
import pandas as pd

In [ ]:
source_lang = 'Hindi'
target_lang = 'Sanskrit'

In [ ]:
language_code = {'Hindi':'hi', 'Sanskrit':'sa', 'Kannada':'kn'}
#s_lang = language_code[FLAGS.source_lang.capitalize()]
#t_lang = language_code[FLAGS.target_lang.capitalize()]
s_lang = 'hi'
t_lang = 'sa'

In [ ]:

# if FLAGS.target_lang.capitalize() == 'Kannada':
#     kannada_sentences = []
#     for sentence in predictions:
#         sentence = UnicodeIndicTransliterator.transliterate(sentence, 'hi', 'kn')
#         kannada_sentences.append(sentence)      

predictions_data = pd.DataFrame({f'{source_lang}':tokenized_datasets['test'][source_lang.capitalize()], 
                                    f'{target_lang}':tokenized_datasets['test'][target_lang.capitalize()],
                                    'predictions':  predictions})

predictions_data.to_csv(f'{source_lang}_{target_lang}_predictions.csv')                                                                                                                                                                       
